<a href="https://colab.research.google.com/github/noushsuon/honours-project/blob/edit-carbon-configuration/NoPretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install comet-ml
!pip install cumulator
!pip install geopy
!pip install GPUtil
!pip install py-cpuinfo
!pip install datasets
!pip install evaluate
!pip install accelerate -U
!pip install codecarbon
!pip install transformers
!pip install tokenizers

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import comet_ml
from comet_ml import Experiment
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

import csv
hardwareFile = "/usr/local/lib/python3.10/dist-packages/codecarbon/data/hardware/cpu_power.csv"
cpu_data = ['13th Gen Intel(R) Core(TM) i9-13950HX', 55]
with open(hardwareFile, 'a', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(cpu_data)

comet_ml.init(api_key='uyaIbaxDfy7jK9j7u4KNxeKNg',project_name='comet-bert-experiment')
experiment = comet_ml.Experiment(api_key="uyaIbaxDfy7jK9j7u4KNxeKNg",project_name="comet-bert-experiment",)

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
[codecarbon INFO @ 18:06:14] [setup] RAM Tracking...
[codecarbon INFO @ 18:06:14] [setup] GPU Tracking...
[codecarbon INFO @ 18:06:14] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:06:14] [setup] CPU Tracking...
[codecarbon WARNING @ 18:06:14] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:06:16] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13950HX
[codecarbon INFO @ 18:06:16] >>> Tracker's metadata:
[codecarbon INFO @ 18:06:16]   Platform system: Linux-5.15.133.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 18:06:16]   Python version: 3.10.12
[codecarbon INFO @ 18:06:16]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 18

In [4]:
import csv
hardwareFile = "/usr/local/lib/python3.10/dist-packages/cumulator/hardware_data/cpu.csv"
cpu_data = ['Core i9-13950HX', 'Raptor Lake-HX', '1 / 23','2.2 to 5.5 GHz','BGA 1964 ','10 nm', '36 MB', '55 W', 55]
with open(hardwareFile, 'a', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(cpu_data)

In [5]:
baseFile = "/usr/local/lib/python3.10/dist-packages/cumulator/base.py"
with open(baseFile, 'r') as file:
        content = file.read()
with open(baseFile, 'w') as file:
        file.write(content.replace("src.cumulator", "cumulator"))
from cumulator import base
cumulator = base.Cumulator()
cumulator.on()

CPU recognized: TDP set to 55


In [6]:
from datasets import load_dataset
import transformers

raw_datasets = load_dataset("squad")
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [7]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizerFast
import os
combined_data = [context + " " + question for context, question in zip(raw_datasets['train']['context'], raw_datasets['train']['question'])]
tokenizer = BertWordPieceTokenizer(clean_text=True,lowercase=False,handle_chinese_chars=False)
tokenizer.train_from_iterator(combined_data, length=512, special_tokens=["[UNK]", "[SEP]", "[CLS]", "[PAD]", "[MASK]"])
path = "/content"
if not os.path.exists(path):
    os.makedirs(path)
tokenizer.save_model(path)
tokenizer = BertTokenizerFast.from_pretrained(path, model_max_length=512, do_lower_case=False)

[codecarbon INFO @ 18:06:31] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 18:06:31] Energy consumed for all GPUs : 0.000119 kWh. Total GPU Power : 28.431377089593916 W
[codecarbon INFO @ 18:06:31] Energy consumed for all CPUs : 0.000115 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 18:06:31] 0.000257 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:06:46] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 18:06:46] Energy consumed for all GPUs : 0.000198 kWh. Total GPU Power : 19.15501366730228 W
[codecarbon INFO @ 18:06:46] Energy consumed for all CPUs : 0.000232 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 18:06:46] 0.000479 kWh of electricity used since the beginning.


In [8]:
tokenizer

BertTokenizerFast(name_or_path='/content', vocab_size=30000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
context = raw_datasets["train"][1902]["context"]
question = raw_datasets["train"][1902]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

"[CLS] What television station made a documentary on Chopin? [SEP] Chopin's life was covered in a BBC TV documentary Chopin – The Women Behind The Music ( 2010 ), and in a 2010 documentary realised by Angelo Bozzolini and Roberto Prosseda for Italian television. [SEP]"

In [10]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second", #only truncates the context
        stride=stride,
        return_overflowing_tokens=True, #model will store strings ignored when truncating
        return_offsets_mapping=True, #stores the location of tokens in original text
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [11]:
#use previously defined function to preprocess training data
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset) #check the sentences have been tokenised

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

[codecarbon INFO @ 18:07:01] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 18:07:01] Energy consumed for all GPUs : 0.000290 kWh. Total GPU Power : 22.65500400493134 W
[codecarbon INFO @ 18:07:01] Energy consumed for all CPUs : 0.000348 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 18:07:01] 0.000710 kWh of electricity used since the beginning.


(87599, 88518)

In [12]:
#similar to training data but with simplified offset mapping, use of example ids and not the start and end positions,
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [13]:
#use function to preprocess validation data
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(10570, 10792)

post processing

In [14]:
#initialise new/untrained model
from transformers import BertConfig, BertForQuestionAnswering
config = BertConfig.from_pretrained("bert-base-cased")
config.vocab_size = len(tokenizer)
model = BertForQuestionAnswering(config)


[codecarbon INFO @ 18:07:16] Energy consumed for RAM : 0.000095 kWh. RAM Power : 5.789747714996338 W


In [15]:
from tqdm.auto import tqdm
import collections
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

import evaluate

metric = evaluate.load("squad")

def get_example(index):
    return tokenized_datasets["test"][index]["text"]

def compute_metrics(start_logits, end_logits, features, examples):
    experiment = comet_ml.get_global_experiment()
    example_to_features = collections.defaultdict(list)

    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []

    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    metric_result = metric.compute(predictions=predicted_answers, references=theoretical_answers)
    if experiment:
        experiment.log_metric("f1", metric_result['f1'])
        experiment.log_metric("exact_match", metric_result['exact_match'])
    return metric_result

[codecarbon INFO @ 18:07:16] Energy consumed for all GPUs : 0.000373 kWh. Total GPU Power : 20.57780207224324 W
[codecarbon INFO @ 18:07:16] Energy consumed for all CPUs : 0.000461 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 18:07:16] 0.000929 kWh of electricity used since the beginning.


Training

In [16]:
import transformers
from transformers import TrainingArguments
import accelerate

%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

args = TrainingArguments(
    "no-pretraining-bert-final",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
    push_to_hub=True,
    report_to=["comet_ml"],
    logging_strategy="epoch",
    seed=42,
    load_best_model_at_end=False,
    save_total_limit=1,
)

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
[codecarbon INFO @ 18:07:18] Energy consumed for RAM : 0.000098 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 18:07:18] Energy consumed for all GPUs : 0.000396 kWh. Total GPU Power : 40.80131154506296 W
[codecarbon INFO @ 18:07:18] Energy consumed for all CPUs : 0.000477 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 18:07:18] 0.000971 kWh of electricity used since the beginning.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: -------------------------------------

Epoch,Training Loss,Validation Loss


[codecarbon INFO @ 18:07:37] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 18:07:37] Energy consumed for all GPUs : 0.000220 kWh. Total GPU Power : 52.84855330618053 W
[codecarbon INFO @ 18:07:37] Energy consumed for all CPUs : 0.000115 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 18:07:37] 0.000360 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:07:52] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 18:07:52] Energy consumed for all GPUs : 0.000454 kWh. Total GPU Power : 56.299256662649725 W
[codecarbon INFO @ 18:07:52] Energy consumed for all CPUs : 0.000230 kWh. Total CPU Power : 27.5 W
[codecarbon INFO @ 18:07:52] 0.000732 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:08:07] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 18:08:07] Energy consumed for all GPUs : 0.000688 kWh. Total GPU Power : 56.2280951360209

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

In [ ]:
cumulator.off()
cumulator.on()

In [ ]:
cumulator.display_carbon_footprint()

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator, DataCollatorForLanguageModeling

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)

eval_dataloader = DataLoader(
    validation_set,
    collate_fn=default_data_collator,
    batch_size=8
)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name
model_name = "no-pretraining-bert-final"
repo_name = get_full_repo_name(model_name)
repo_name

In [ ]:
from huggingface_hub import Repository
model_checkpoint = 'no-pretraining-bert-final'
output_dir = "no-pretraining-bert-final-accelerate" + model_checkpoint

repo = Repository(output_dir, clone_from=repo_name)

In [ ]:
#finetune cycle
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )

    for key, value in metrics.items():
        experiment.log_metric(key, value)

    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [ ]:
experiment.end()

In [ ]:
cumulator.off()
cumulator.display_carbon_footprint()

In [ ]:
cumulator.time_list